In [1]:
# EDA LLM Assistant - Comprehensive Exploratory Data Analysis

"""
This notebook provides a complete EDA framework with automated analysis capabilities.
It combines traditional EDA techniques with modern automated profiling tools.
"""

# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path
import os

# Configuration
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

print("📊 EDA LLM Assistant Initialized!")
print("=" * 50)

Matplotlib is building the font cache; this may take a moment.


📊 EDA LLM Assistant Initialized!


In [4]:
# Data Loading and Initial Setup

class EDAAssistant:
    """
    A comprehensive EDA assistant class that provides automated analysis capabilities
    """
    
    def __init__(self):
        self.data = None
        self.numeric_columns = []
        self.categorical_columns = []
        self.datetime_columns = []
        
    def load_data(self, file_path, table_name=None, query=None, **kwargs):
        """
        Load data from various file formats including SQLite databases
        
        Parameters:
        file_path: Path to the data file
        table_name: For SQLite databases, specify the table name to load
        query: For SQLite databases, specify a custom SQL query
        **kwargs: Additional parameters for pandas read functions
        """
        file_extension = Path(file_path).suffix.lower()
        
        try:
            if file_extension == '.csv':
                self.data = pd.read_csv(file_path, **kwargs)
            elif file_extension in ['.xlsx', '.xls']:
                self.data = pd.read_excel(file_path, **kwargs)
            elif file_extension == '.json':
                self.data = pd.read_json(file_path, **kwargs)
            elif file_extension == '.parquet':
                self.data = pd.read_parquet(file_path, **kwargs)
            elif file_extension in ['.db', '.sqlite', '.sqlite3']:
                self.data = self._load_sqlite_data(file_path, table_name, query, **kwargs)
            else:
                raise ValueError(f"Unsupported file format: {file_extension}")
                
            print(f"✅ Data loaded successfully!")
            print(f"Shape: {self.data.shape}")
            self._analyze_column_types()
            return self.data
            
        except Exception as e:
            print(f"❌ Error loading data: {str(e)}")
            return None
    
    def _load_sqlite_data(self, file_path, table_name=None, query=None, **kwargs):
        """Load data from SQLite database"""
        import sqlite3
        
        # Create connection to SQLite database
        conn = sqlite3.connect(file_path)
        
        try:
            if query:
                # Use custom query
                print(f"🔍 Executing custom query...")
                data = pd.read_sql_query(query, conn, **kwargs)
            elif table_name:
                # Load specific table
                print(f"📊 Loading table: {table_name}")
                data = pd.read_sql_query(f"SELECT * FROM {table_name}", conn, **kwargs)
            else:
                # If no table specified, show available tables and load the first one
                tables = self._get_sqlite_tables(conn)
                if not tables:
                    raise ValueError("No tables found in the SQLite database")
                
                print(f"📋 Available tables: {tables}")
                table_name = tables[0]
                print(f"📊 Loading first table: {table_name}")
                data = pd.read_sql_query(f"SELECT * FROM {table_name}", conn, **kwargs)
            
            return data
            
        finally:
            conn.close()
    
    def _get_sqlite_tables(self, conn):
        """Get list of tables in SQLite database"""
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = [row[0] for row in cursor.fetchall()]
        return tables
    
    def explore_sqlite_database(self, file_path):
        """Explore SQLite database structure"""
        import sqlite3
        
        conn = sqlite3.connect(file_path)
        
        try:
            print("🗄️  SQLITE DATABASE EXPLORATION")
            print("=" * 50)
            
            # Get all tables
            tables = self._get_sqlite_tables(conn)
            print(f"📋 Tables found: {len(tables)}")
            
            for table in tables:
                print(f"\n📊 TABLE: {table}")
                
                # Get table info
                cursor = conn.cursor()
                cursor.execute(f"PRAGMA table_info({table})")
                columns_info = cursor.fetchall()
                
                print("   Columns:")
                for col_info in columns_info:
                    col_name, col_type = col_info[1], col_info[2]
                    print(f"     - {col_name} ({col_type})")
                
                # Get row count
                cursor.execute(f"SELECT COUNT(*) FROM {table}")
                row_count = cursor.fetchone()[0]
                print(f"   Rows: {row_count:,}")
                
                # Show sample data
                cursor.execute(f"SELECT * FROM {table} LIMIT 3")
                sample_data = cursor.fetchall()
                if sample_data:
                    print("   Sample data (first 3 rows):")
                    column_names = [col_info[1] for col_info in columns_info]
                    sample_df = pd.DataFrame(sample_data, columns=column_names)
                    print(sample_df.to_string(index=False))
            
            return tables
            
        except Exception as e:
            print(f"❌ Error exploring database: {str(e)}")
            return []
        finally:
            conn.close()
    
    def _analyze_column_types(self):
        """Analyze and categorize column types"""
        self.numeric_columns = self.data.select_dtypes(include=[np.number]).columns.tolist()
        self.categorical_columns = self.data.select_dtypes(include=['object', 'category']).columns.tolist()
        self.datetime_columns = self.data.select_dtypes(include=['datetime64']).columns.tolist()
        
        print(f"📊 Column Analysis:")
        print(f"   Numeric columns: {len(self.numeric_columns)}")
        print(f"   Categorical columns: {len(self.categorical_columns)}")
        print(f"   DateTime columns: {len(self.datetime_columns)}")

# Initialize EDA Assistant
eda = EDAAssistant()

# Example data loading (uncomment and modify path as needed)
# eda.load_data('your_dataset.csv')

print("🔧 EDA Assistant class created successfully!")

🔧 EDA Assistant class created successfully!


In [ ]:
# SQLite Database Support and Examples

def create_sample_sqlite_database(db_path="sample_data.db"):
    """Create a sample SQLite database for demonstration"""
    import sqlite3
    
    # Generate sample data
    sample_data = generate_sample_data(1000)
    
    # Create SQLite database
    conn = sqlite3.connect(db_path)
    
    try:
        # Create main customers table
        customers_data = sample_data[['age', 'income', 'score', 'category', 'city', 'is_premium', 'rating']].copy()
        customers_data.to_sql('customers', conn, if_exists='replace', index_label='customer_id')
        
        # Create transactions table (derived from customers)
        np.random.seed(42)
        transactions_data = []
        for customer_id in range(len(customers_data)):
            # Generate 1-5 transactions per customer
            num_transactions = np.random.randint(1, 6)
            for _ in range(num_transactions):
                transaction = {
                    'customer_id': customer_id,
                    'transaction_date': sample_data.iloc[customer_id]['registration_date'] + pd.Timedelta(days=np.random.randint(0, 365)),
                    'amount': np.random.exponential(100),
                    'product_category': np.random.choice(['Electronics', 'Clothing', 'Books', 'Home', 'Sports'])
                }
                transactions_data.append(transaction)
        
        transactions_df = pd.DataFrame(transactions_data)
        transactions_df.to_sql('transactions', conn, if_exists='replace', index_label='transaction_id')
        
        # Create products table
        products_data = pd.DataFrame({
            'product_name': ['Laptop', 'Smartphone', 'Headphones', 'T-Shirt', 'Jeans', 
                           'Novel', 'Cookbook', 'Chair', 'Table', 'Basketball'],
            'category': ['Electronics', 'Electronics', 'Electronics', 'Clothing', 'Clothing',
                        'Books', 'Books', 'Home', 'Home', 'Sports'],
            'price': [999.99, 699.99, 199.99, 29.99, 79.99, 
                     14.99, 24.99, 149.99, 299.99, 39.99],
            'in_stock': [True, True, False, True, True, True, True, False, True, True]
        })
        products_data.to_sql('products', conn, if_exists='replace', index_label='product_id')
        
        print(f"✅ Sample SQLite database created: {db_path}")
        print("📋 Tables created:")
        print("   • customers (1000 rows) - Customer demographics and ratings")
        print("   • transactions (~3000 rows) - Purchase transactions")
        print("   • products (10 rows) - Product catalog")
        
        return db_path
        
    finally:
        conn.close()

def demonstrate_sqlite_loading():
    """Demonstrate different ways to load SQLite data"""
    print("🗄️  SQLITE DATA LOADING EXAMPLES")
    print("=" * 60)
    
    # Create sample database
    db_path = create_sample_sqlite_database()
    
    print("\n📖 LOADING METHODS:")
    print("1. Explore database structure:")
    print("   eda.explore_sqlite_database('sample_data.db')")
    
    print("\n2. Load entire table:")
    print("   eda.load_data('sample_data.db', table_name='customers')")
    
    print("\n3. Load with custom SQL query:")
    print("   query = 'SELECT * FROM customers WHERE age > 30 AND rating >= 4'")
    print("   eda.load_data('sample_data.db', query=query)")
    
    print("\n4. Load with JOIN across tables:")
    print("   query = '''")
    print("   SELECT c.age, c.income, c.city, t.amount, t.product_category")
    print("   FROM customers c")
    print("   JOIN transactions t ON c.customer_id = t.customer_id")
    print("   WHERE c.is_premium = 1")
    print("   '''")
    print("   eda.load_data('sample_data.db', query=query)")
    
    print("\n5. Auto-load first table (if no table specified):")
    print("   eda.load_data('sample_data.db')")
    
    return db_path

# Add SQLite methods to EDA Assistant
def add_sqlite_methods():
    EDAAssistant.explore_sqlite_database = lambda self, file_path: self.explore_sqlite_database(file_path)

add_sqlite_methods()

print("✅ SQLite support added to EDA Assistant!")
print("\n💡 SQLite Features:")
print("• Automatic table detection and listing")
print("• Support for custom SQL queries")
print("• Database structure exploration")
print("• JOIN operations across multiple tables")
print("• Supports .db, .sqlite, .sqlite3 file extensions")

In [ ]:
# 🚀 SQLite DEMONSTRATION: Run this cell to see SQLite support in action!

print("🗄️  DEMONSTRATING SQLITE DATABASE FUNCTIONALITY")
print("=" * 60)

# Create and demonstrate SQLite loading
db_path = demonstrate_sqlite_loading()

print("\n" + "="*30 + " LIVE DEMO " + "="*30)

# Create a new EDA instance for SQLite demo
eda_sqlite = EDAAssistant()

print("\n1️⃣ EXPLORING DATABASE STRUCTURE:")
tables = eda_sqlite.explore_sqlite_database(db_path)

print("\n2️⃣ LOADING CUSTOMERS TABLE:")
eda_sqlite.load_data(db_path, table_name='customers')
print(f"Loaded data shape: {eda_sqlite.data.shape}")
print("First 3 rows:")
print(eda_sqlite.data.head(3))

print("\n3️⃣ LOADING WITH CUSTOM QUERY - High-value customers:")
query = """
SELECT customer_id, age, income, score, rating, city
FROM customers 
WHERE income > 50000 AND rating >= 4
ORDER BY income DESC
LIMIT 10
"""

eda_sqlite.load_data(db_path, query=query)
print(f"High-value customers data shape: {eda_sqlite.data.shape}")
print(eda_sqlite.data)

print("\n4️⃣ COMPLEX JOIN QUERY - Customer transactions analysis:")
complex_query = """
SELECT 
    c.age,
    c.income,
    c.city,
    c.rating,
    COUNT(t.transaction_id) as transaction_count,
    AVG(t.amount) as avg_transaction_amount,
    SUM(t.amount) as total_spent
FROM customers c
LEFT JOIN transactions t ON c.customer_id = t.customer_id
GROUP BY c.customer_id, c.age, c.income, c.city, c.rating
HAVING transaction_count > 2
ORDER BY total_spent DESC
LIMIT 15
"""

print("Loading customer transaction analysis...")
eda_sqlite.load_data(db_path, query=complex_query)
print(f"Analysis data shape: {eda_sqlite.data.shape}")
print(eda_sqlite.data)

print("\n" + "="*60)
print("✅ SQLite demonstration completed!")
print("🎯 You can now run EDA analysis on the SQLite data:")
print("   eda_sqlite.complete_eda_analysis()")
print("   eda_sqlite.correlation_analysis()")
print("   eda_sqlite.plot_numeric_distributions()")
print("="*60)

In [ ]:
# Basic Data Exploration Methods

def basic_info(data):
    """Display basic information about the dataset"""
    print("📋 DATASET OVERVIEW")
    print("=" * 50)
    print(f"Shape: {data.shape}")
    print(f"Memory usage: {data.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    print("\n🔍 DATA TYPES:")
    print(data.dtypes)
    
    print("\n📊 MISSING VALUES:")
    missing = data.isnull().sum()
    missing_percent = (missing / len(data)) * 100
    missing_df = pd.DataFrame({
        'Column': missing.index,
        'Missing Count': missing.values,
        'Missing %': missing_percent.values
    }).sort_values('Missing %', ascending=False)
    print(missing_df[missing_df['Missing Count'] > 0])
    
    print("\n🎯 UNIQUE VALUES:")
    unique_counts = data.nunique().sort_values(ascending=False)
    print(unique_counts)
    
    return missing_df

def statistical_summary(data, numeric_columns):
    """Generate statistical summary for numeric columns"""
    print("\n📈 STATISTICAL SUMMARY")
    print("=" * 50)
    
    if numeric_columns:
        stats = data[numeric_columns].describe()
        print(stats)
        
        # Additional statistics
        print("\n📊 ADDITIONAL STATISTICS:")
        additional_stats = pd.DataFrame({
            'Skewness': data[numeric_columns].skew(),
            'Kurtosis': data[numeric_columns].kurtosis(),
            'Variance': data[numeric_columns].var()
        })
        print(additional_stats)
    else:
        print("No numeric columns found for statistical analysis.")

# Add methods to EDA Assistant class
def add_basic_methods():
    EDAAssistant.basic_info = lambda self: basic_info(self.data)
    EDAAssistant.statistical_summary = lambda self: statistical_summary(self.data, self.numeric_columns)

add_basic_methods()
print("✅ Basic exploration methods added to EDA Assistant!")

In [ ]:
# Data Visualization Functions

def plot_missing_values(data):
    """Visualize missing values pattern"""
    missing = data.isnull().sum()
    missing = missing[missing > 0].sort_values(ascending=False)
    
    if len(missing) > 0:
        plt.figure(figsize=(12, 6))
        
        # Missing values bar plot
        plt.subplot(1, 2, 1)
        missing.plot(kind='bar', color='coral')
        plt.title('Missing Values Count by Column')
        plt.xlabel('Columns')
        plt.ylabel('Missing Count')
        plt.xticks(rotation=45)
        
        # Missing values heatmap
        plt.subplot(1, 2, 2)
        sns.heatmap(data[missing.index].isnull(), cbar=True, cmap='viridis')
        plt.title('Missing Values Heatmap')
        plt.xticks(rotation=45)
        
        plt.tight_layout()
        plt.show()
    else:
        print("🎉 No missing values found in the dataset!")

def plot_numeric_distributions(data, numeric_columns, max_cols=3):
    """Plot distributions of numeric columns"""
    if not numeric_columns:
        print("No numeric columns to plot.")
        return
    
    n_cols = min(max_cols, len(numeric_columns))
    n_rows = (len(numeric_columns) + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(5*n_cols, 4*n_rows))
    axes = axes.flatten() if n_rows > 1 else [axes] if n_rows == 1 else axes
    
    for i, col in enumerate(numeric_columns):
        if i < len(axes):
            # Histogram with KDE
            data[col].hist(bins=30, alpha=0.7, ax=axes[i], density=True)
            data[col].plot(kind='kde', ax=axes[i], color='red')
            axes[i].set_title(f'Distribution of {col}')
            axes[i].set_xlabel(col)
            axes[i].set_ylabel('Density')
    
    # Hide empty subplots
    for i in range(len(numeric_columns), len(axes)):
        axes[i].set_visible(False)
    
    plt.tight_layout()
    plt.show()

def plot_categorical_distributions(data, categorical_columns, max_cols=2, top_n=10):
    """Plot distributions of categorical columns"""
    if not categorical_columns:
        print("No categorical columns to plot.")
        return
    
    n_cols = min(max_cols, len(categorical_columns))
    n_rows = (len(categorical_columns) + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(8*n_cols, 4*n_rows))
    if len(categorical_columns) == 1:
        axes = [axes]
    else:
        axes = axes.flatten() if n_rows > 1 else axes
    
    for i, col in enumerate(categorical_columns):
        if i < len(axes):
            # Get top N categories
            value_counts = data[col].value_counts().head(top_n)
            
            # Bar plot
            value_counts.plot(kind='bar', ax=axes[i], color='skyblue')
            axes[i].set_title(f'Top {min(top_n, len(value_counts))} values in {col}')
            axes[i].set_xlabel(col)
            axes[i].set_ylabel('Count')
            axes[i].tick_params(axis='x', rotation=45)
    
    # Hide empty subplots
    for i in range(len(categorical_columns), len(axes)):
        axes[i].set_visible(False)
    
    plt.tight_layout()
    plt.show()

# Add visualization methods to EDA Assistant
def add_visualization_methods():
    EDAAssistant.plot_missing_values = lambda self: plot_missing_values(self.data)
    EDAAssistant.plot_numeric_distributions = lambda self, max_cols=3: plot_numeric_distributions(self.data, self.numeric_columns, max_cols)
    EDAAssistant.plot_categorical_distributions = lambda self, max_cols=2, top_n=10: plot_categorical_distributions(self.data, self.categorical_columns, max_cols, top_n)

add_visualization_methods()
print("✅ Visualization methods added to EDA Assistant!")

In [ ]:
# Correlation Analysis and Advanced Visualizations

def correlation_analysis(data, numeric_columns):
    """Perform correlation analysis on numeric columns"""
    if len(numeric_columns) < 2:
        print("Need at least 2 numeric columns for correlation analysis.")
        return
    
    print("🔗 CORRELATION ANALYSIS")
    print("=" * 50)
    
    # Calculate correlation matrix
    corr_matrix = data[numeric_columns].corr()
    
    # Display high correlations
    high_corr = []
    for i in range(len(corr_matrix.columns)):
        for j in range(i+1, len(corr_matrix.columns)):
            if abs(corr_matrix.iloc[i, j]) > 0.7:
                high_corr.append({
                    'Feature 1': corr_matrix.columns[i],
                    'Feature 2': corr_matrix.columns[j],
                    'Correlation': corr_matrix.iloc[i, j]
                })
    
    if high_corr:
        print("⚡ HIGH CORRELATIONS (|r| > 0.7):")
        high_corr_df = pd.DataFrame(high_corr)
        print(high_corr_df.sort_values('Correlation', key=abs, ascending=False))
    else:
        print("No high correlations found (|r| > 0.7)")
    
    # Plot correlation heatmap
    plt.figure(figsize=(12, 8))
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0,
                square=True, fmt='.2f', cbar_kws={"shrink": .8}, mask=mask)
    plt.title('Correlation Heatmap')
    plt.tight_layout()
    plt.show()
    
    return corr_matrix

def outlier_detection(data, numeric_columns):
    """Detect outliers using IQR method"""
    print("🚨 OUTLIER DETECTION")
    print("=" * 50)
    
    outlier_counts = {}
    
    for col in numeric_columns:
        Q1 = data[col].quantile(0.25)
        Q3 = data[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        outliers = data[(data[col] < lower_bound) | (data[col] > upper_bound)]
        outlier_counts[col] = len(outliers)
    
    outlier_df = pd.DataFrame(list(outlier_counts.items()), 
                             columns=['Column', 'Outlier Count'])
    outlier_df['Outlier %'] = (outlier_df['Outlier Count'] / len(data)) * 100
    outlier_df = outlier_df.sort_values('Outlier Count', ascending=False)
    
    print(outlier_df)
    
    # Plot outliers
    if len(numeric_columns) > 0:
        plt.figure(figsize=(15, 4))
        data[numeric_columns].boxplot(ax=plt.gca())
        plt.title('Box Plots for Outlier Detection')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
    
    return outlier_df

# Add advanced analysis methods
def add_advanced_methods():
    EDAAssistant.correlation_analysis = lambda self: correlation_analysis(self.data, self.numeric_columns)
    EDAAssistant.outlier_detection = lambda self: outlier_detection(self.data, self.numeric_columns)

add_advanced_methods()
print("✅ Advanced analysis methods added to EDA Assistant!")

In [ ]:
# Automated EDA with ydata-profiling (Pandas Profiling)

def automated_eda_report(data, title="Automated EDA Report"):
    """Generate automated EDA report using ydata-profiling"""
    try:
        from ydata_profiling import ProfileReport
        
        print("🤖 Generating Automated EDA Report...")
        print("This may take a few minutes for large datasets...")
        
        # Create profile report
        profile = ProfileReport(
            data, 
            title=title,
            explorative=True,
            minimal=False
        )
        
        # Display in notebook
        profile.to_notebook_iframe()
        
        # Save HTML report (optional)
        # profile.to_file("eda_report.html")
        
        return profile
        
    except ImportError:
        print("❌ ydata-profiling not installed. Install with: pip install ydata-profiling")
        return None
    except Exception as e:
        print(f"❌ Error generating automated report: {str(e)}")
        return None

def sweetviz_analysis(data, target_column=None):
    """Generate EDA report using Sweetviz"""
    try:
        import sweetviz as sv
        
        print("🍭 Generating Sweetviz EDA Report...")
        
        # Create report
        if target_column and target_column in data.columns:
            report = sv.analyze(data, target_feat=target_column)
        else:
            report = sv.analyze(data)
        
        # Show report
        report.show_notebook()
        
        return report
        
    except ImportError:
        print("❌ sweetviz not installed. Install with: pip install sweetviz")
        return None
    except Exception as e:
        print(f"❌ Error generating Sweetviz report: {str(e)}")
        return None

# Add automated EDA methods
def add_automated_methods():
    EDAAssistant.automated_eda_report = lambda self, title="Automated EDA Report": automated_eda_report(self.data, title)
    EDAAssistant.sweetviz_analysis = lambda self, target_column=None: sweetviz_analysis(self.data, target_column)

add_automated_methods()
print("✅ Automated EDA methods added to EDA Assistant!")

In [ ]:
# Feature Engineering and Analysis

def feature_engineering_insights(data, numeric_columns, categorical_columns):
    """Provide insights for feature engineering"""
    print("🔧 FEATURE ENGINEERING INSIGHTS")
    print("=" * 50)
    
    insights = []
    
    # Check for skewed numeric features
    print("📊 SKEWNESS ANALYSIS:")
    skewness = data[numeric_columns].skew().abs().sort_values(ascending=False)
    highly_skewed = skewness[skewness > 1]
    if len(highly_skewed) > 0:
        print(f"Highly skewed features (|skew| > 1): {list(highly_skewed.index)}")
        insights.append("Consider log transformation for highly skewed features")
    else:
        print("No highly skewed features found")
    
    # Check for high cardinality categorical features
    print("\n🎯 CARDINALITY ANALYSIS:")
    high_cardinality = []
    for col in categorical_columns:
        unique_ratio = data[col].nunique() / len(data)
        if unique_ratio > 0.5:
            high_cardinality.append((col, data[col].nunique(), unique_ratio))
    
    if high_cardinality:
        print("High cardinality categorical features:")
        for col, nunique, ratio in high_cardinality:
            print(f"  {col}: {nunique} unique values ({ratio:.2%} of total)")
        insights.append("Consider grouping rare categories or using embedding techniques")
    else:
        print("No high cardinality categorical features found")
    
    # Check for potential date features
    print("\n📅 POTENTIAL DATE FEATURES:")
    date_candidates = []
    for col in data.columns:
        if data[col].dtype == 'object':
            sample_values = data[col].dropna().head(10).astype(str)
            if any(len(str(val)) in [8, 10, 19] and any(c in str(val) for c in ['-', '/', ':']) for val in sample_values):
                date_candidates.append(col)
    
    if date_candidates:
        print(f"Potential date columns: {date_candidates}")
        insights.append("Consider converting date strings to datetime and extracting features")
    else:
        print("No obvious date columns found")
    
    # Feature interaction suggestions
    print("\n🔗 FEATURE INTERACTION SUGGESTIONS:")
    if len(numeric_columns) >= 2:
        insights.append("Consider creating ratio features between numeric columns")
        insights.append("Consider polynomial features for important numeric features")
    
    print("\n💡 RECOMMENDATIONS:")
    for i, insight in enumerate(insights, 1):
        print(f"{i}. {insight}")
    
    return insights

def data_quality_assessment(data):
    """Assess data quality issues"""
    print("🔍 DATA QUALITY ASSESSMENT")
    print("=" * 50)
    
    quality_issues = []
    
    # Duplicate rows
    duplicates = data.duplicated().sum()
    if duplicates > 0:
        print(f"⚠️  Duplicate rows: {duplicates} ({duplicates/len(data)*100:.2f}%)")
        quality_issues.append(f"Remove {duplicates} duplicate rows")
    else:
        print("✅ No duplicate rows found")
    
    # Constant columns
    constant_cols = [col for col in data.columns if data[col].nunique() <= 1]
    if constant_cols:
        print(f"⚠️  Constant columns: {constant_cols}")
        quality_issues.append(f"Consider removing constant columns: {constant_cols}")
    else:
        print("✅ No constant columns found")
    
    # High missing value columns
    missing_threshold = 0.5
    high_missing = data.isnull().mean()
    high_missing_cols = high_missing[high_missing > missing_threshold].index.tolist()
    if high_missing_cols:
        print(f"⚠️  High missing value columns (>{missing_threshold*100}%): {high_missing_cols}")
        quality_issues.append(f"Consider dropping or imputing high missing columns")
    else:
        print(f"✅ No columns with >{missing_threshold*100}% missing values")
    
    # Mixed data types in object columns
    print("\n🔄 MIXED DATA TYPE CHECK:")
    for col in data.select_dtypes(include=['object']).columns:
        sample = data[col].dropna().head(100)
        numeric_count = sum(pd.to_numeric(sample, errors='coerce').notna())
        if 0 < numeric_count < len(sample):
            print(f"⚠️  {col}: Mixed numeric/text data")
            quality_issues.append(f"Review mixed data types in column: {col}")
    
    print(f"\n📝 Quality Issues Found: {len(quality_issues)}")
    for i, issue in enumerate(quality_issues, 1):
        print(f"{i}. {issue}")
    
    return quality_issues

# Add feature engineering methods
def add_feature_methods():
    EDAAssistant.feature_engineering_insights = lambda self: feature_engineering_insights(self.data, self.numeric_columns, self.categorical_columns)
    EDAAssistant.data_quality_assessment = lambda self: data_quality_assessment(self.data)

add_feature_methods()
print("✅ Feature engineering methods added to EDA Assistant!")

In [ ]:
# Time Series Analysis (if applicable)

def time_series_analysis(data, datetime_columns):
    """Analyze time series data if available"""
    if not datetime_columns:
        print("No datetime columns found for time series analysis.")
        return
    
    print("📅 TIME SERIES ANALYSIS")
    print("=" * 50)
    
    for col in datetime_columns:
        print(f"\n🕒 Analyzing {col}:")
        
        # Basic info
        date_range = data[col].max() - data[col].min()
        print(f"   Date range: {data[col].min()} to {data[col].max()}")
        print(f"   Duration: {date_range}")
        
        # Create time-based features
        data[f'{col}_year'] = data[col].dt.year
        data[f'{col}_month'] = data[col].dt.month
        data[f'{col}_day'] = data[col].dt.day
        data[f'{col}_dayofweek'] = data[col].dt.dayofweek
        data[f'{col}_hour'] = data[col].dt.hour if hasattr(data[col].dt, 'hour') else None
        
        # Plot time series
        plt.figure(figsize=(12, 6))
        
        # Count by date
        date_counts = data[col].dt.date.value_counts().sort_index()
        plt.subplot(2, 2, 1)
        date_counts.plot(kind='line')
        plt.title(f'Records Count Over Time - {col}')
        plt.xticks(rotation=45)
        
        # Monthly pattern
        monthly_counts = data[f'{col}_month'].value_counts().sort_index()
        plt.subplot(2, 2, 2)
        monthly_counts.plot(kind='bar', color='lightgreen')
        plt.title(f'Records by Month - {col}')
        plt.xlabel('Month')
        
        # Day of week pattern
        dow_counts = data[f'{col}_dayofweek'].value_counts().sort_index()
        plt.subplot(2, 2, 3)
        dow_counts.plot(kind='bar', color='orange')
        plt.title(f'Records by Day of Week - {col}')
        plt.xlabel('Day of Week (0=Monday)')
        
        # Hourly pattern (if applicable)
        if f'{col}_hour' in data.columns and data[f'{col}_hour'].notna().any():
            hourly_counts = data[f'{col}_hour'].value_counts().sort_index()
            plt.subplot(2, 2, 4)
            hourly_counts.plot(kind='bar', color='red')
            plt.title(f'Records by Hour - {col}')
            plt.xlabel('Hour')
        
        plt.tight_layout()
        plt.show()

# Add time series methods
def add_time_series_methods():
    EDAAssistant.time_series_analysis = lambda self: time_series_analysis(self.data, self.datetime_columns)

add_time_series_methods()
print("✅ Time series analysis methods added to EDA Assistant!")

In [ ]:
# Complete EDA Workflow

def complete_eda_analysis(eda_assistant, target_column=None):
    """
    Run complete EDA analysis workflow
    
    Parameters:
    eda_assistant: EDAAssistant instance with loaded data
    target_column: Target column for supervised learning (optional)
    """
    if eda_assistant.data is None:
        print("❌ No data loaded. Please load data first using eda.load_data()")
        return
    
    print("🚀 STARTING COMPLETE EDA ANALYSIS")
    print("=" * 70)
    
    # 1. Basic Information
    print("\n" + "="*20 + " STEP 1: BASIC INFORMATION " + "="*20)
    eda_assistant.basic_info()
    
    # 2. Statistical Summary
    print("\n" + "="*20 + " STEP 2: STATISTICAL SUMMARY " + "="*20)
    eda_assistant.statistical_summary()
    
    # 3. Missing Values Visualization
    print("\n" + "="*20 + " STEP 3: MISSING VALUES " + "="*20)
    eda_assistant.plot_missing_values()
    
    # 4. Distribution Analysis
    print("\n" + "="*20 + " STEP 4: DISTRIBUTION ANALYSIS " + "="*20)
    eda_assistant.plot_numeric_distributions()
    eda_assistant.plot_categorical_distributions()
    
    # 5. Correlation Analysis
    print("\n" + "="*20 + " STEP 5: CORRELATION ANALYSIS " + "="*20)
    eda_assistant.correlation_analysis()
    
    # 6. Outlier Detection
    print("\n" + "="*20 + " STEP 6: OUTLIER DETECTION " + "="*20)
    eda_assistant.outlier_detection()
    
    # 7. Feature Engineering Insights
    print("\n" + "="*20 + " STEP 7: FEATURE ENGINEERING " + "="*20)
    eda_assistant.feature_engineering_insights()
    
    # 8. Data Quality Assessment
    print("\n" + "="*20 + " STEP 8: DATA QUALITY " + "="*20)
    eda_assistant.data_quality_assessment()
    
    # 9. Time Series Analysis (if applicable)
    print("\n" + "="*20 + " STEP 9: TIME SERIES ANALYSIS " + "="*20)
    eda_assistant.time_series_analysis()
    
    # 10. Target Variable Analysis (if provided)
    if target_column and target_column in eda_assistant.data.columns:
        print("\n" + "="*20 + f" STEP 10: TARGET ANALYSIS ({target_column}) " + "="*20)
        target_analysis(eda_assistant.data, target_column, eda_assistant.numeric_columns, eda_assistant.categorical_columns)
    
    print("\n" + "="*70)
    print("✅ COMPLETE EDA ANALYSIS FINISHED!")
    print("💡 Consider running automated EDA for additional insights:")
    print("   eda.automated_eda_report()")
    print("   eda.sweetviz_analysis()")
    print("=" * 70)

def target_analysis(data, target_column, numeric_columns, categorical_columns):
    """Analyze relationship between features and target variable"""
    print(f"🎯 TARGET VARIABLE: {target_column}")
    print("=" * 50)
    
    target_type = 'numeric' if target_column in numeric_columns else 'categorical'
    print(f"Target type: {target_type}")
    print(f"Target distribution:")
    if target_type == 'categorical':
        print(data[target_column].value_counts())
    else:
        print(data[target_column].describe())
    
    # Visualize target
    plt.figure(figsize=(12, 4))
    
    if target_type == 'categorical':
        plt.subplot(1, 2, 1)
        data[target_column].value_counts().plot(kind='bar', color='lightblue')
        plt.title(f'Distribution of {target_column}')
        plt.xticks(rotation=45)
        
        plt.subplot(1, 2, 2)
        data[target_column].value_counts().plot(kind='pie', autopct='%1.1f%%')
        plt.title(f'Proportion of {target_column}')
    else:
        plt.subplot(1, 2, 1)
        data[target_column].hist(bins=30, alpha=0.7, color='lightblue')
        plt.title(f'Distribution of {target_column}')
        
        plt.subplot(1, 2, 2)
        data[target_column].plot(kind='box', color='lightblue')
        plt.title(f'Box plot of {target_column}')
    
    plt.tight_layout()
    plt.show()
    
    # Feature-target relationships
    print(f"\n🔗 FEATURE-TARGET RELATIONSHIPS:")
    
    # Numeric features vs target
    numeric_features = [col for col in numeric_columns if col != target_column]
    if numeric_features and target_type == 'numeric':
        # Correlation with target
        correlations = data[numeric_features + [target_column]].corr()[target_column].drop(target_column)
        correlations = correlations.abs().sort_values(ascending=False)
        print("Top correlations with target:")
        print(correlations.head(10))
        
        # Plot top correlations
        top_features = correlations.head(4).index
        if len(top_features) > 0:
            fig, axes = plt.subplots(2, 2, figsize=(12, 8))
            axes = axes.flatten()
            for i, feature in enumerate(top_features):
                if i < 4:
                    data.plot.scatter(x=feature, y=target_column, ax=axes[i], alpha=0.6)
                    axes[i].set_title(f'{feature} vs {target_column}')
            plt.tight_layout()
            plt.show()

# Add complete workflow methods
def add_workflow_methods():
    EDAAssistant.complete_eda_analysis = lambda self, target_column=None: complete_eda_analysis(self, target_column)

add_workflow_methods()
print("✅ Complete EDA workflow methods added!")

In [ ]:
# Sample Data Generation and Usage Example

def generate_sample_data(n_samples=1000):
    """Generate sample data for testing the EDA assistant"""
    np.random.seed(42)
    
    # Generate sample data
    data = {
        'age': np.random.normal(35, 12, n_samples).astype(int),
        'income': np.random.lognormal(10, 0.8, n_samples),
        'score': np.random.beta(2, 5, n_samples) * 100,
        'category': np.random.choice(['A', 'B', 'C', 'D'], n_samples, p=[0.4, 0.3, 0.2, 0.1]),
        'city': np.random.choice(['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix'], n_samples),
        'is_premium': np.random.choice([True, False], n_samples, p=[0.3, 0.7]),
        'registration_date': pd.date_range('2020-01-01', '2023-12-31', periods=n_samples),
        'rating': np.random.choice([1, 2, 3, 4, 5], n_samples, p=[0.05, 0.1, 0.3, 0.35, 0.2])
    }
    
    # Create DataFrame
    df = pd.DataFrame(data)
    
    # Add some missing values
    missing_indices = np.random.choice(df.index, size=int(0.1 * n_samples), replace=False)
    df.loc[missing_indices, 'income'] = np.nan
    
    missing_indices = np.random.choice(df.index, size=int(0.05 * n_samples), replace=False)
    df.loc[missing_indices, 'score'] = np.nan
    
    # Add some outliers
    outlier_indices = np.random.choice(df.index, size=int(0.02 * n_samples), replace=False)
    df.loc[outlier_indices, 'age'] = np.random.choice([120, 150, 200], size=len(outlier_indices))
    
    return df

print("✅ Sample data generation function created!")
print("\n📖 USAGE INSTRUCTIONS:")
print("=" * 50)
print("1. Load your data:")
print("   eda = EDAAssistant()")
print("   eda.load_data('your_dataset.csv')")
print("\n2. Or generate sample data for testing:")
print("   sample_data = generate_sample_data()")
print("   eda.data = sample_data")
print("   eda._analyze_column_types()")
print("\n3. Run complete EDA analysis:")
print("   eda.complete_eda_analysis(target_column='rating')")
print("\n4. Or run individual analyses:")
print("   eda.basic_info()")
print("   eda.plot_missing_values()")
print("   eda.correlation_analysis()")
print("   eda.outlier_detection()")
print("\n5. Generate automated reports:")
print("   eda.automated_eda_report()")
print("   eda.sweetviz_analysis(target_column='rating')")

In [ ]:
# 🚀 DEMONSTRATION: Run this cell to see the EDA Assistant in action!

# Generate sample data
print("🎲 Generating sample data...")
sample_data = generate_sample_data(1000)

# Initialize EDA Assistant with sample data
eda.data = sample_data
eda._analyze_column_types()

print(f"\n✅ Sample dataset created with {len(sample_data)} rows and {len(sample_data.columns)} columns")
print(f"Columns: {list(sample_data.columns)}")

# Display first few rows
print("\n👀 First 5 rows of sample data:")
print(sample_data.head())

print("\n🎯 Now you can run any EDA analysis:")
print("• eda.complete_eda_analysis(target_column='rating') - Full analysis")
print("• eda.basic_info() - Basic information")
print("• eda.correlation_analysis() - Correlation analysis") 
print("• eda.plot_missing_values() - Missing values visualization")
print("• eda.outlier_detection() - Outlier detection")
print("• eda.automated_eda_report() - Automated pandas profiling report")

print("\n" + "="*50)
print("🎉 EDA LLM Assistant is ready to use!")
print("Try running: eda.complete_eda_analysis(target_column='rating')")
print("="*50)

In [ ]:
# 📚 EDA Assistant Documentation and Additional Features

print("📚 EDA LLM ASSISTANT - COMPREHENSIVE DOCUMENTATION")
print("=" * 60)

print("\n🎯 MAIN FEATURES:")
print("1. Automated Data Loading - Supports CSV, Excel, JSON, Parquet, SQLite")
print("2. SQLite Database Support - Table exploration, custom queries, JOINs")
print("3. Basic Data Exploration - Shape, types, missing values, unique counts")
print("4. Statistical Analysis - Descriptive statistics, skewness, kurtosis")
print("5. Visualization Suite - Distributions, correlations, missing values")
print("6. Correlation Analysis - Heatmaps and high correlation detection")
print("7. Outlier Detection - IQR method with box plots")
print("8. Feature Engineering Insights - Skewness, cardinality, date detection")
print("9. Data Quality Assessment - Duplicates, constants, mixed types")
print("10. Time Series Analysis - Temporal patterns and feature extraction")
print("11. Automated EDA Reports - Pandas Profiling and Sweetviz integration")

print("\n🔧 AVAILABLE METHODS:")
methods = [
    "load_data(file_path, table_name, query) - Load data from file or SQLite",
    "explore_sqlite_database(file_path) - Explore SQLite database structure",
    "basic_info() - Display basic dataset information",
    "statistical_summary() - Show statistical summary",
    "plot_missing_values() - Visualize missing values",
    "plot_numeric_distributions() - Plot numeric distributions",
    "plot_categorical_distributions() - Plot categorical distributions",
    "correlation_analysis() - Analyze correlations",
    "outlier_detection() - Detect outliers",
    "feature_engineering_insights() - Get feature engineering tips",
    "data_quality_assessment() - Assess data quality",
    "time_series_analysis() - Analyze time series data",
    "automated_eda_report() - Generate pandas profiling report",
    "sweetviz_analysis() - Generate Sweetviz report",
    "complete_eda_analysis() - Run full EDA workflow"
]

for method in methods:
    print(f"• {method}")

print("\n💡 QUICK START EXAMPLES:")
print("```python")
print("# Initialize")
print("eda = EDAAssistant()")
print("")
print("# Load CSV/Excel/JSON/Parquet")
print("eda.load_data('your_data.csv')")
print("")
print("# Load SQLite database")
print("eda.explore_sqlite_database('database.db')  # Explore structure first")
print("eda.load_data('database.db', table_name='customers')  # Load specific table")
print("eda.load_data('database.db', query='SELECT * FROM customers WHERE age > 30')")
print("")
print("# Run complete analysis")
print("eda.complete_eda_analysis(target_column='your_target')")
print("```")

print("\n🎨 CUSTOMIZATION OPTIONS:")
print("• Modify visualization parameters (colors, figure sizes)")
print("• Adjust thresholds for outlier detection")
print("• Customize automated report settings")
print("• Add custom analysis functions")

print("\n📊 SUPPORTED FILE FORMATS:")
print("• CSV (.csv)")
print("• Excel (.xlsx, .xls)")
print("• JSON (.json)")
print("• Parquet (.parquet)")
print("• SQLite (.db, .sqlite, .sqlite3)")
print("")
print("📈 SUPPORTED DATA TYPES:")
print("• Numeric: int, float, complex")
print("• Categorical: object, category")
print("• Temporal: datetime64")
print("• Boolean: bool")

print("\n🚀 PERFORMANCE TIPS:")
print("• For large datasets (>100K rows), consider sampling first")
print("• Use automated reports sparingly on very wide datasets")
print("• Filter columns of interest before analysis")
print("• Save reports to HTML for sharing")

print("\n" + "=" * 60)
print("🎉 Happy Exploring! Your data insights await!")
print("=" * 60)